In [1]:
import litho1pt0 as litho
from pprint import pprint
import numpy as np

%matplotlib inline

In [2]:
## make a global raster of some quantity

lonv, latv = np.meshgrid(  np.linspace(-180.0,180.0,360), np.linspace(-89.999,89.999,360), indexing='xy')
l1 = litho.layer_depth(lat=latv, lon=lonv, layerID="LID-BOTTOM")
#l1 = litho.layer_depth(lat=latv, lon=lonv, layerID="CRUST3-BOTTOM")

In [3]:
earth_radius = 6371e3

l1 = (earth_radius - l1) / earth_radius

In [4]:
from scipy.interpolate import NearestNDInterpolator

In [5]:
x = np.deg2rad(lonv.flatten())
y = np.deg2rad(latv.flatten())
z = l1.flatten()

pts = list(zip(x,y))

f = NearestNDInterpolator(pts, z)

In [6]:
import gmsh

In [7]:
gmsh.initialize()

In [8]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.05)

In [9]:
gmsh.model.add("SphereTopo")

In [10]:
sphere = gmsh.model.occ.add_sphere(0,0,0, 1.0)

In [11]:
gmsh.model.occ.synchronize()

In [12]:
gmsh.model.mesh.generate(2)

Info    : Meshing 1D...
Info    : [ 40%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000314881s, CPU 0.002846s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.374147s, CPU 0.664716s)
Info    : 6092 nodes 12245 elements


In [13]:
#gmsh.fltk.run()

In [14]:
nodeTags = {}
nodeCoords = {}
elementTypes = {}
elementTags = {}
elementNodeTags = {}


In [15]:
entities = gmsh.model.get_entities()

In [16]:
# get the nodes and elements
for e in entities:
    nodeTags[e], nodeCoords[e], _ = gmsh.model.mesh.getNodes(e[0], e[1])
    elementTypes[e], elementTags[e], elementNodeTags[e] = gmsh.model.mesh.getElements(e[0], e[1])

In [17]:
gmsh.model.mesh.clear()

In [18]:
import numpy as np
import random

for e in entities: 
    for i in range(2, len(nodeCoords[e]), 3):
        x = nodeCoords[e][i-2]
        y = nodeCoords[e][i-1]
        z = nodeCoords[e][i]
        rho = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arctan2(y, x)
        phi = np.arcsin(z / rho)
        rho *= f(theta, phi)
        nodeCoords[e][i-2] = rho * np.cos(phi) * np.cos(theta)
        nodeCoords[e][i-1] = rho * np.cos(phi) * np.sin(theta)
        nodeCoords[e][i] = rho * np.sin(phi)
    gmsh.model.mesh.addNodes(e[0], e[1], nodeTags[e], nodeCoords[e])
    gmsh.model.mesh.addElements(e[0], e[1], elementTypes[e], elementTags[e],
                               elementNodeTags[e])

In [19]:
gmsh.write("test.vtk")

Info    : Writing 'test.vtk'...
Info    : Done writing 'test.vtk'


In [20]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.10.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220501
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tet

Info    : Optimization starts (volume = 3.97995) with worst = 0.00097811 / average = 0.784062:
Info    : 0.00 < quality < 0.10 :       276 elements
Info    : 0.10 < quality < 0.20 :       848 elements
Info    : 0.20 < quality < 0.30 :      1620 elements
Info    : 0.30 < quality < 0.40 :      2478 elements
Info    : 0.40 < quality < 0.50 :      4021 elements
Info    : 0.50 < quality < 0.60 :      6547 elements
Info    : 0.60 < quality < 0.70 :     13657 elements
Info    : 0.70 < quality < 0.80 :     32310 elements
Info    : 0.80 < quality < 0.90 :     56214 elements
Info    : 0.90 < quality < 1.00 :     28892 elements
Info    : 2710 edge swaps, 88 node relocations (volume = 3.97995): worst = 0.172085 / average = 0.79556 (Wall 0.0889976s, CPU 0.089335s)
Info    : 2727 edge swaps, 92 node relocations (volume = 3.97995): worst = 0.29103 / average = 0.795628 (Wall 0.115146s, CPU 0.11544s)
Info    : 2728 edge swaps, 92 node relocations (volume = 3.97995): worst = 0.298048 / average = 0.79563

Info    : Writing '/home/romain/PROJECTS/Project_plate_boundaries/test.vtk'...
Info    : Done writing '/home/romain/PROJECTS/Project_plate_boundaries/test.vtk'
